In [1]:
import random 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import Bunch
from sklearn.model_selection import train_test_split
from pathlib import Path

from skimage.io import imread
from skimage.transform import resize

%load_ext autoreload
%autoreload 2

In [2]:
# Function to read the input data and store in n-dimensional array format
import numpy as np
import os

# read all the images from the folder structure (images are separated in different folders according to classes)
def load_files(container_path,dimension=(150, 150,3)):
    image_dir = Path(container_path)
    folders = [directory for directory in image_dir.iterdir() if directory.is_dir()]
    categories = [fo.name for fo in folders]
    descr = "Cataract dataset"
    images = []
    flat_data = []
    target = []
    for i, direc in enumerate(folders):
        for file in direc.iterdir():
            img = imread(file)
            img_resized = resize(img, dimension,anti_aliasing_sigma=True)
            flat_data.append(img_resized.flatten()) 
            images.append(img_resized)
            ll=i
            if i==0:
                ll=-1
            target.append(ll)
    flat_data = np.array(flat_data)
    target = np.array(target)
    images = np.array(images)
    
    return Bunch(data=flat_data,
                 target=target,
                 target_names=categories,
                 images=images,
                 DESCR=descr)

In [3]:
# two classes normal, cataract

def read_data():
    # path to hog modified eye images
    image_dir ='\dataset_hog'
    image_dataset=load_files(image_dir)
    # split the data into training an testing data and labels (30% of data is for testing)
    X_train, X_test, y_train, y_test = train_test_split(image_dataset.data, image_dataset.target, test_size=0.3,random_state=109)
    return X_train, y_train, X_test, y_test


# Invoke the above functions to get our data
X_train, y_train, X_test, y_test = read_data()

# check the shapes of tarin and test data
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)

print ('Test data shape: ', X_test.shape)
print ('Test labels shape: ', y_test.shape)

C:\Users\Bharati\Anaconda3\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Bharati\Anaconda3\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Train data shape:  (6991, 67500)
Train labels shape:  (6991,)
Test data shape:  (2997, 67500)
Test labels shape:  (2997,)


In [4]:
# function to train the svm model
def svm_fit(epochs,c,X,Y):
    m,n_features=X.shape[0],X.shape[1]
    w=np.zeros(n_features)
    #print(Y)
    #print(X.shape)
    #print(len(X))
    i=1
    while i<epochs:
        j = np.random.choice(m, 1)[0]
        eta=1. / (c*(i+1))
        x, y = X[j], Y[j]
        score = w.dot(x)
        if y*score < 1:
            w = (1 - eta*c)*w + eta*y*x
        else:
            w = (1 - eta*c)*w
        i+=1
    return w

In [5]:
# function to predict the image classes based on the learned weight matrix
def svm_predict(X,w):
    y_pred=[]
    i=0
    while i < len(X):
        xi=X[i]
        #dp=0
        score = w.dot(xi)
        if score>=0:
            y_pred.append(1)
        else:
            y_pred.append(-1)
        i+=1
    return y_pred

In [10]:
from sklearn import metrics

# start training the model
num_itrs=1
epochs=500
c=0.015
#for i in range(num_itrs):
w=svm_fit(epochs,c,X_train,y_train)

In [12]:
# test the model and display classification report
y_pred=svm_predict(X_test,w)
print("Classification report - \n",
    metrics.classification_report(y_test, y_pred))
#print('Training accuracy: %f' % (np.mean(y_test == y_pred)))

Classification report - 
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        58
           1       0.98      1.00      0.99      2939

   micro avg       0.98      0.98      0.98      2997
   macro avg       0.49      0.50      0.50      2997
weighted avg       0.96      0.98      0.97      2997



In [ ]:
# read the images to predict
image_dir = '\predict'
image_dataset=load_files(image_dir)
X_1, X_2, y_1, y_2 = train_test_split(image_dataset.data, image_dataset.target, test_size=0.0,random_state=109)

# predict the image classes
y_pred_1=svm_predict(X_1,w)

# class -1 belongs to cataract and class 1 belongs to normal
for i in y_pred_1: 
    if i==-1:
        print("Cataract")
    else:
        print("Normal")